# Install the package

First, you need to install the package from github and import it:

In [ ]:
# Install the bulkhours package
!rm -rf bulkhours && git clone https://github.com/guydegnol/bulkhours.git

# import the bulkhours package
import bulkhours

# Generate header links
bulkhours.generate_header_links("examples/1_Simple_Exercice_Automation.ipynb")

Cloning into 'bulkhours'...
remote: Enumerating objects: 5159, done.
remote: Counting objects: 100% (758/758), done.
remote: Compressing objects: 100% (368/368), done.
remote: Total 5159 (delta 474), reused 642 (delta 361), pack-reused 4401
Receiving objects: 100% (5159/5159), 40.86 MiB | 10.84 MiB/s, done.
Resolving deltas: 100% (3311/3311), done.



[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/guydegnol/bulkhours/blob/main/examples/1_Simple_Exercice_Automation.ipynb) [![Open In Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/guydegnol/bulkhours/blob/main/examples/1_Simple_Exercice_Automation.ipynb) [![GitHub](https://badgen.net/badge/icon/Open%20in%20Github?icon=github&label)](https://github.com/guydegnol/bulkhours/blob/main/examples/1_Simple_Exercice_Automation.ipynb) [![Visual Studio](https://badgen.net/badge/icon/Open%20in%20Visual%20Studio?icon=visualstudio&label)](https://vscode.dev/github/guydegnol/bulkhours/blob/main/examples/1_Simple_Exercice_Automation.ipynb) [![CC-0 license](https://img.shields.io/badge/License-CC--0-blue.svg)](https://creativecommons.org/licenses/by-nd/4.0)
                            

In [ ]:
# Calcul des effets de serre des planetes telluriques
from bulkhours import constants as bkc

planetes = bkc.DataFrame(index=["mercure", "venus", "terre", "mars"], columns=["Albedo", "d_ua", "T_C"])
bkc.print(r"Equilibre thermique: $(1-A)\frac{ L_\odot R^2}{4 d^2} \equiv (1-S) 4 \pi R^2 \sigma T^4$", size="+1")
planetes["T_C/no atmo."] = (((1-planetes.Albedo) * bkc.L_soleil / (16 * np.pi * (planetes.d_ua * bkc.d_terresoleil)**2 * bkc.sigma))**0.25-bkc.c2k).round()
planetes["Effet de serre"] = (1-( (1 - planetes.Albedo) * bkc.L_soleil / (planetes.T_C + bkc.c2k)**4 / (16 * bkc.pi * (planetes.d_ua * bkc.d_terresoleil)**2 * bkc.sigma))).round(3)

bkc.print(r"Forcage radiatif: $F = S\cdot \sigma \cdot T^4$", size="+1")
planetes["Forcage radiatif (W/m2)"] = (planetes["Effet de serre"] * bkc.sigma * (planetes.T_C + bkc.c2k)**4).round(3)
display(planetes)

# Affichage des propriétés de la terre (pour cc(CO2)=280ppm)

# Affichage de quelques scenarios
fig, ax = plt.subplots(1, ncols=2, figsize=(12, 4))
def calculate_T(ppm):
    F = bkc.sigma * bkc.S_terre *(bkc.T_terre + bkc.c2k)**4
    F_prime = F * np.log(ppm) / np.log(280)
    T_prime = ( (1-bkc.S_terre) * (bkc.T_terre + bkc.c2k)**4 + F_prime / bkc.sigma)**0.25 - bkc.c2k
    return T_prime-15

co2 = bulkhours.get_data("co2.concentrations", index="year").loc[1900:].ffill()["CO2 concentrations"]
co2.plot(legend=True, ax=ax[0])
calculate_T(co2).plot(legend=True, ax=ax[1], label="Anomalie de Temperature °C (Holocène)")
def scenario(label, damping_trend):
    with ipywidgets.Output():
        y_pred = smtsa.ExponentialSmoothing(co2, initialization_method='legacy-heuristic', damped_trend=True,
                                            trend="mul").fit(smoothing_trend=30, damping_trend=damping_trend).forecast(77); y_pred.index += 1900
    y_pred.plot(legend=True, ax=ax[0], label=label)
    calculate_T(y_pred).plot(legend=True, ax=ax[1], label=label)

scenario("Scenario 1", 0.987)
scenario("Scenario 2", 0.97)
scenario("Scenario 3", 0.95)